In [1]:
from pyspark.mllib.recommendation import ALS
from numpy import array
import math

In [66]:
import pandas as pd
df = sc.textFile('file:/vagrant/triplets_file.txt').map(lambda x: x.split("\t"))
df.zipWithIndex().take(5)
df_songs = pd.read_csv('file:/vagrant/song_data.csv')
df.columns = ['user', 'song', 'listens']


[[u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOAKIMP12A8C130995', u'1'],
 [u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOBBMDR12A8C13253B', u'2'],
 [u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOBXHDL12A81C204C0', u'1'],
 [u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOBYHAJ12A6701BF1D', u'1'],
 [u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SODACBL12A8C13C273', u'1']]

In [3]:
df_songs.head()

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [68]:
df.zipWithIndex().take(5)

[([u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOAKIMP12A8C130995', u'1'],
  0),
 ([u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOBBMDR12A8C13253B', u'2'],
  1),
 ([u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOBXHDL12A81C204C0', u'1'],
  2),
 ([u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOBYHAJ12A6701BF1D', u'1'],
  3),
 ([u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SODACBL12A8C13C273', u'1'],
  4)]

In [4]:
from sklearn import preprocessing

user_id_le = preprocessing.LabelEncoder()
song_id_le = preprocessing.LabelEncoder()

user_id_le.fit(df.user)
song_id_le.fit(df_songs.song_id)
df['user_1'] = user_id_le.transform(df.user)
df_songs['song_id_1']= song_id_le.transform(df_songs.song_id)

,user,song,listens,user_1
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0,2542
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,2542
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,2542
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0,2542
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0,2542
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5.0,2542
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODXRTY12AB0180F3B,1.0,2542
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFGUAY12AB017B0A8,1.0,2542
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFRQTD12A81C233C0,1.0,2542
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOHQWYZ12A6D4FA701,1.0,2542


In [6]:
user_mapping=df[['user','user_1']].copy().drop_duplicates()
df_2=df.merge(df_songs,left_on='song',right_on='song_id')
df_2.head(2)

,user,song,listens,user_1,song_id,title,release,artist_name,year,song_id_1
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0,2542,SOAKIMP12A8C130995,The Cove,Thicker Than Water,Jack Johnson,0,17370
1,7c86176941718984fed11b7c0674ff04c029b480,SOAKIMP12A8C130995,1.0,1695,SOAKIMP12A8C130995,The Cove,Thicker Than Water,Jack Johnson,0,17370


In [7]:
songs_rating_final=df_2[['user_1','song_id_1','listens']].copy()

In [8]:
pdDF = sqlContext.createDataFrame(songs_rating_final)
songs_rating = pdDF.rdd

In [9]:
songs_rating = songs_rating.cache()

In [10]:
df_songs_1=df_songs[['song_id_1','title','release','artist_name','year']].copy()
df_songs_1[['song_id_1','title','release','artist_name','year']]=df_songs_1[['song_id_1','title','release','artist_name','year']].astype(str)
df_song_mapping=df_songs[['song_id','song_id_1']].copy().drop_duplicates()


In [11]:
pdDF_songs = sqlContext.createDataFrame(df_songs_1)
songs_data = pdDF_songs.rdd

pdDF_song_map = sqlContext.createDataFrame(df_song_mapping)
songs_data_map = pdDF_song_map.rdd

In [12]:
songs_data = songs_data.cache()
songs_data_map=songs_data_map.cache()

In [13]:
training_RDD, validation_RDD, test_RDD = songs_rating.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [14]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5L
iterations = 10
regularization_parameter = 0.1
alpha = 0.05
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.trainImplicit(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter,alpha=alpha)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 4 the RMSE is 8.52762756132
For rank 8 the RMSE is 8.52518805403
For rank 12 the RMSE is 8.52311070008
The best model was trained with rank 12


In [15]:
#predictions.take(3)

In [16]:
#rates_and_preds.take(3)

In [17]:
model = ALS.trainImplicit(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter,alpha=alpha)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'For testing data the RMSE is %s' % (error)

For testing data the RMSE is 7.09578789978


In [18]:
from pyspark.mllib.recommendation import MatrixFactorizationModel

model_path = os.path.join('file:/vagrant', 'model', 'songs')

# Save and load model
#model.save(sc, model_path)
same_model = MatrixFactorizationModel.load(sc, model_path)

In [19]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

songs_ID_with_ratings_RDD = (songs_rating.map(lambda x: (x[1], x[2])).groupByKey())
songs_ID_with_avg_ratings_RDD = songs_ID_with_ratings_RDD.map(get_counts_and_averages)
songs_rating_counts_RDD = songs_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

In [20]:
#songs_rating.take(2)

In [21]:
user_mapping[user_mapping['user']=='ff4322e94814d3c7895d07e6f94139b092862611']['user_1'].values[0]

3554

In [22]:
user_id = 1417
user_id_songs_listened = songs_rating.filter(lambda x: x[0] == user_id)
user_id_songs_listened.take(1)

[Row(user_1=1417, song_id_1=442793, listens=1.0)]

In [23]:
songs_listened_by_user = user_id_songs_listened.map(lambda x: x[1], user_id_songs_listened)
songs_listened_by_user_1=[x for x in songs_listened_by_user.toLocalIterator()]

In [24]:
#songs_data_map.take(1)

In [25]:
user_not_listened_songs_RDD = songs_data_map.filter(lambda x: x[1] not in songs_listened_by_user_1)\
.map(lambda x: (user_id, x[1]))
#new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_songs_RDD)

In [26]:
user_recommendations_RDD = same_model.predictAll(user_not_listened_songs_RDD)

In [27]:
user_recommendations_song_RDD = user_recommendations_RDD.map(lambda x: (x.product, x.rating))

In [28]:
song_titles = songs_data.map(lambda x: (int(x[0]),x[1] + ': ' + x[3] + '-' + x[4]))
#song_titles.take(1)

In [29]:
user_recommendations_rating_title_and_count_RDD = \
    user_recommendations_song_RDD.join(song_titles).join(songs_rating_counts_RDD)

In [30]:
user_recommendations_rating_title_and_count_RDD = \
    user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

In [31]:
top_songs = user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=50).distinct().takeOrdered(5, key=lambda x: -x[1])

print ('TOP recommended songs (with more than 50 listen count):\n%s' %
        '\n'.join(map(str, top_songs)))

TOP recommended songs (with more than 50 listen count):
(u'Howlin\x19 For You: The Black Keys-2010', 0.017589063048736822, 64)
(u'Tighten Up: The Black Keys-2010', 0.01754664945756507, 97)
(u'Cosmic Love: Florence + The Machine-2009', 0.015783973320884034, 105)
(u'Bubble Toes: Jack Johnson-2000', 0.012751606438180633, 150)
(u"I'm Not Calling You A Liar: Florence + The Machine-2009", 0.011349329743872783, 58)


In [54]:
import difflib
import random
import requests
import ipywidgets as widgets
from ipywidgets import Button, Layout
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, Text
from IPython.display import clear_output, display
import time

user_list = pd.read_csv('file:/vagrant/songProfiles.csv')
user_list['songs'] = user_list['songs'].str.split('|')

# Button for generating the recommendations

def on_rec_clicked(rec):
    clear_output()
    print('Loading Recommendations: ............')
    st_time = datetime.datetime.now()
    user_id = user_mapping[user_mapping['user']==user_list['user_id'][tab.selected_index]]['user_1'].values[0]
    user_id_songs_listened = songs_rating.filter(lambda x: x[0] == user_id)
    songs_listened_by_user = user_id_songs_listened.map(lambda x: x[1], user_id_songs_listened)
    songs_listened_by_user_1=[x for x in songs_listened_by_user.toLocalIterator()]
    user_not_listened_songs_RDD = songs_data_map.filter(lambda x: x[1] not in songs_listened_by_user_1)\
    .map(lambda x: (user_id, x[1]))
    
    user_recommendations_RDD = same_model.predictAll(user_not_listened_songs_RDD)
    user_recommendations_song_RDD = user_recommendations_RDD.map(lambda x: (x.product, x.rating))
    song_titles = songs_data.map(lambda x: (int(x[0]),x[1] + ': ' + x[3]))
    top_songs = \
        user_recommendations_song_RDD.join(song_titles).join(songs_rating_counts_RDD)\
        .map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))\
        .filter(lambda r: r[2]>=50).distinct().takeOrdered(10, key=lambda x: -x[1])
    print('Recommendation took {} '.format(datetime.datetime.now() - st_time))
    display(pd.DataFrame({'songs':[row[0] for row in top_songs]}))
    #results = is_model.get_user_items(child.description)
    #clear_output()
    #display(results[['rank','song']])

rec = Button(description='Generate  Recommendations',
       layout=Layout(width='100%', height='80px'), button_style = 'info')

rec.on_click(on_rec_clicked)
    
#Clear Button
clear = Button(description='Clear',
           layout=Layout(width='100%', height='80px'),button_style = 'primary')

def on_clear(clear):
    clear_output()
    return(tab)

clear.on_click(on_clear)


tab_list = user_list['user_name']
children = []
tab = widgets.Tab()
for i in range(len(tab_list)):
    #tab.children = (dropdown,button_form)
    tab.set_title(i, str(tab_list[i]))
    songs = user_list['songs'][i]
    user_song_labels= [widgets.Label(str(song)) for song in songs]
    child = widgets.VBox(user_song_labels,description = user_list['user_id'][i],
            layout=Layout(
               display='flex',
             flex_flow='column',
               border='solid 2px',
               width='75%'))

    button_form = Box(children=(rec,clear),
                    layout=Layout(display='flex',
                     flex_flow='column',
                    border='solid 2px',
                    fill='grey',          
           align_items='center',
           width='50%'))
    full_form = widgets.HBox([child,button_form])
    children.append(full_form)
    


tab.children = tuple(children)
tab

Loading Recommendations: ............
Recommendation took 0:02:05.053876 


,songs
0,Bring Me To Life: Evanescence
1,You're The One: Dwight Yoakam
2,Nothin' On You [feat. Bruno Mars] (Album Versi...
3,My Immortal: Evanescence
4,My Immortal (Album Version): Evanescence
5,Bubble Toes: Jack Johnson
6,Gives You Hell: The All-American Rejects
7,Hips Don't Lie: Shakira ft. Wyclef Jean
8,Hips Don't Lie (featuring Wyclef Jean): Shakir...
9,Livin' On A Prayer: Bon Jovi


In [56]:
user_list['user_id'][tab.selected_index]['user_1'].values[0]

TypeError: string indices must be integers, not str

In [57]:
 user_mapping[user_mapping['user']==user_list['user_id'][tab.selected_index]]['user_1'].values[0]

3529

In [58]:
user_list['user_id'][tab.selected_index]

'fd7b78f7c22a39b64bbb9e49cee0ffe2dac5b849'